# This file will look at cleaning the dataframe containing the sales information 

In [1]:
#Import any required dependencies 
import pandas as pd 
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
sales_df = pd.read_csv("../data/vgsales.csv")
sales_df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


### Looking at the above we can see there are several columns that are not related to our analytical purposes. We must remove those columns 

In [16]:
#Remove the rank, name, genre and publisher columns
step_one = sales_df.drop(columns=['Rank', 'Genre', 'Publisher'], axis=1)
step_one = pd.DataFrame(step_one)

In [17]:
#Display the dataframe
step_one.head()

,Name,Platform,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Wii Sports,Wii,2006.0,41.49,29.02,3.77,8.46,82.74
1,Super Mario Bros.,NES,1985.0,29.08,3.58,6.81,0.77,40.24
2,Mario Kart Wii,Wii,2008.0,15.85,12.88,3.79,3.31,35.82
3,Wii Sports Resort,Wii,2009.0,15.75,11.01,3.28,2.96,33.00
4,Pokemon Red/Pokemon Blue,GB,1996.0,11.27,8.89,10.22,1.00,31.37


### We have our simplified dataframe for cleaning. Now we must decide on what years to look at for the sales trends. As the findings of this project are going to help people find the best gaming platform, we want to look at the years closest to the present year, 2021. As the Year column is already in a float64 format, we do not need to convert it into another type to rank the years from the most recent to the latest 

In [18]:
#Rank the years column from highest to lowest to find the years to analyse 

    #Grab the years column and find all the unique values within it 
years = sales_df["Year"].unique()
    #Use nump to sort the above array into decending order 
years_sorted = np.sort(years)[::-1]
    #Display the results 
years_sorted

array([  nan, 2020., 2017., 2016., 2015., 2014., 2013., 2012., 2011.,
       2010., 2009., 2008., 2007., 2006., 2005., 2004., 2003., 2002.,
       2001., 2000., 1999., 1998., 1997., 1996., 1995., 1994., 1993.,
       1992., 1991., 1990., 1989., 1988., 1987., 1986., 1985., 1984.,
       1983., 1982., 1981., 1980.])

### Looking at the above we can see that 2020 is the year that is closest to the present year. However, the year after that is 2017. There is a two year gap. The 2020 data cannot be used in creating a reliable analysis for trends over time. Therefore, the data for sales will look at 2017, 2016, 2015, and 2014. 


In [29]:
### The next thing we want to do is refine our dataframe even further and for it to contain only the years we want to analyse. The other years are of no importance to our findings/ data story 
step_two_pt1 = step_one.drop(step_one[step_one.Year <2012].index)
step_two = step_two_pt1.drop(step_two_pt1[step_two_pt1.Year == 2020].index)
step_two.head()

,Name,Platform,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
16,Grand Theft Auto V,PS3,2013.0,7.01,9.27,0.97,4.14,21.40
23,Grand Theft Auto V,X360,2013.0,9.63,5.31,0.06,1.38,16.38
32,Pokemon X/Pokemon Y,3DS,2013.0,5.17,4.05,4.34,0.79,14.35
33,Call of Duty: Black Ops 3,PS4,2015.0,5.77,5.81,0.35,2.31,14.24
34,Call of Duty: Black Ops II,PS3,2012.0,4.99,5.88,0.65,2.52,14.03


### Just quickly looking back to when we ranked the years to find the most recent years, the value "nan" appears in the array of all the years in the dataset. NaN- also known as "Not a Number"- values will not have been removed in the above step. We must do this in the next step to ensure clean data is available to us 

In [30]:
#Remove all rows that contain a NaN value in the Years column
step_three = step_two[step_two['Year'].notna()]
step_three.head()

,Name,Platform,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
16,Grand Theft Auto V,PS3,2013.0,7.01,9.27,0.97,4.14,21.40
23,Grand Theft Auto V,X360,2013.0,9.63,5.31,0.06,1.38,16.38
32,Pokemon X/Pokemon Y,3DS,2013.0,5.17,4.05,4.34,0.79,14.35
33,Call of Duty: Black Ops 3,PS4,2015.0,5.77,5.81,0.35,2.31,14.24
34,Call of Duty: Black Ops II,PS3,2012.0,4.99,5.88,0.65,2.52,14.03


In [31]:
#Check that all NaN are removed from the years column 
check_years = step_three["Year"].unique()
check_years

array([2013., 2015., 2012., 2014., 2016., 2017.])

### We can further clean the data to only contain the platforms that pertain to our investigation

### The platforms we need to extract are PlayStation 4, XboxOne, PlayStation Vita WiiU and PC. The issue with this dataset is that they are listed as acroynms and the other two datasets list the platforms by their names. The first step we will need to do is extract these acronyms and make a dataframe that only consists of the platforms that are relevant 

In [32]:
#Use ".loc()" to filter the dataframe to now only consist of relevant Platforms 
step_four = step_three.loc[step_three["Platform"].isin(["XOne","PS4","WiiU","PC","PSV"]), :]
#Display the dataframe
step_four

,Name,Platform,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
33,Call of Duty: Black Ops 3,PS4,2015.0,5.77,5.81,0.35,2.31,14.24
44,Grand Theft Auto V,PS4,2014.0,3.80,5.81,0.36,2.02,11.98
77,FIFA 16,PS4,2015.0,1.11,6.06,0.06,1.26,8.49
92,Star Wars Battlefront (2015),PS4,2015.0,2.93,3.29,0.22,1.23,7.67
93,Call of Duty: Advanced Warfare,PS4,2014.0,2.80,3.30,0.14,1.37,7.60
...,...,...,...,...,...,...,...,...
16566,End of Nations,PC,2012.0,0.01,0.00,0.00,0.00,0.01
16569,Resident Evil 4 HD,XOne,2016.0,0.01,0.00,0.00,0.00,0.01
16570,Farming 2017 - The Simulation,PS4,2016.0,0.00,0.01,0.00,0.00,0.01
16576,Rugby Challenge 3,XOne,2016.0,0.00,0.01,0.00,0.00,0.01


In [33]:
#Confirm the that the results of the platform column are what is required
platforms_check = step_four["Platform"].unique()
platforms_check

array(['PS4', 'XOne', 'WiiU', 'PC', 'PSV'], dtype=object)

### The next step we need to take is to rename some of the values in the Platform column so that it can be used as a primary key in a SQL database. This is beacause this dataframe uses the abbreviated version of the platform name where as the other two use their fullnames. 

In [34]:
#Use the ".replace()" function to change the necessary values
step_four["Platform"].replace({"PS4": "PlayStation4", "XOne":"XboxOne", "PSV":"PlayStationVita"}, inplace=True)

In [35]:
#Display the dataframe
step_four

,Name,Platform,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
33,Call of Duty: Black Ops 3,PlayStation4,2015.0,5.77,5.81,0.35,2.31,14.24
44,Grand Theft Auto V,PlayStation4,2014.0,3.80,5.81,0.36,2.02,11.98
77,FIFA 16,PlayStation4,2015.0,1.11,6.06,0.06,1.26,8.49
92,Star Wars Battlefront (2015),PlayStation4,2015.0,2.93,3.29,0.22,1.23,7.67
93,Call of Duty: Advanced Warfare,PlayStation4,2014.0,2.80,3.30,0.14,1.37,7.60
...,...,...,...,...,...,...,...,...
16566,End of Nations,PC,2012.0,0.01,0.00,0.00,0.00,0.01
16569,Resident Evil 4 HD,XboxOne,2016.0,0.01,0.00,0.00,0.00,0.01
16570,Farming 2017 - The Simulation,PlayStation4,2016.0,0.00,0.01,0.00,0.00,0.01
16576,Rugby Challenge 3,XboxOne,2016.0,0.00,0.01,0.00,0.00,0.01


### Finally check for any NaN values in the rows and if they are present, drop them from the dataframe entirely. Once this has been done, reset the index values and export the data to the "cleaned_data" folder 

In [36]:
#Check for NaN in the entire dataframe using ".isnull()" and ".any()"
step_four.isnull().values.any()

False

In [37]:
#There are no NaN values in the dataframe. The index can now be reset 
step_five = step_four.reset_index(drop=True)
step_five

,Name,Platform,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Call of Duty: Black Ops 3,PlayStation4,2015.0,5.77,5.81,0.35,2.31,14.24
1,Grand Theft Auto V,PlayStation4,2014.0,3.80,5.81,0.36,2.02,11.98
2,FIFA 16,PlayStation4,2015.0,1.11,6.06,0.06,1.26,8.49
3,Star Wars Battlefront (2015),PlayStation4,2015.0,2.93,3.29,0.22,1.23,7.67
4,Call of Duty: Advanced Warfare,PlayStation4,2014.0,2.80,3.30,0.14,1.37,7.60
...,...,...,...,...,...,...,...,...
1312,End of Nations,PC,2012.0,0.01,0.00,0.00,0.00,0.01
1313,Resident Evil 4 HD,XboxOne,2016.0,0.01,0.00,0.00,0.00,0.01
1314,Farming 2017 - The Simulation,PlayStation4,2016.0,0.00,0.01,0.00,0.00,0.01
1315,Rugby Challenge 3,XboxOne,2016.0,0.00,0.01,0.00,0.00,0.01


In [39]:
#Export the dataframe as a csv to the "cleaned_data" folder
step_five.to_csv("../cleaned_data/sales.csv", index=False)